In [7]:
import pandas as pd
import numpy as np
from sklearn import preprocessing


In [8]:
df_train = pd.read_csv("./data/train.csv")

In [9]:
drop_df = df_train.drop(['UniqueID'], axis=1)

In [10]:
# Converting to months
drop_df['CREDIT.HISTORY.LENGTH'] = drop_df['CREDIT.HISTORY.LENGTH'].map(lambda ini: (int(ini.split(' ')[0][:-3])*12)+(int(ini.split(' ')[1][:-3])))
# Converting to months
drop_df['AVERAGE.ACCT.AGE'] = drop_df['AVERAGE.ACCT.AGE'].map(lambda ini: (int(ini.split(' ')[0][:-3])*12)+(int(ini.split(' ')[1][:-3])))
# fill empty values
drop_df['Employment.Type'].fillna('NA', inplace=True)

In [11]:
drop_df['Employment.Type'].value_counts()

Self employed    127635
Salaried          97858
NA                 7661
Name: Employment.Type, dtype: int64

In [12]:
# map category to int
drop_df['Employment.Type'] = drop_df['Employment.Type'].map({'Self employed':0,'Salaried':1, 'NA':3})

In [13]:
# label encode scores
le = preprocessing.LabelEncoder()
s = le.fit_transform(drop_df['PERFORM_CNS.SCORE.DESCRIPTION'])

drop_df['PERFORM_CNS.SCORE.DESCRIPTION'] = s

In [14]:
# Create a new feature AGE
drop_df['Age'] = (pd.to_datetime('today').year - pd.to_datetime(drop_df['Date.of.Birth'].values).year)

f = lambda x: x if x>0 else x*(-1)
drop_df['Age'] = drop_df['Age'].map(f)


year_today = pd.to_datetime('today').year
month_today = pd.to_datetime('today').month

current_mon = ((int(str(year_today)[-2:])) * 12 )+ int(month_today)

# convert months passed from date of disbursal 
drop_df['DisbursalInMonths'] = current_mon - drop_df['DisbursalDate'].map(lambda ini: ((int(ini.split('-')[2])*12)+(int(ini.split('-')[1]))))
final_df = drop_df.drop(['DisbursalDate','Date.of.Birth'], axis=1)
final_df.rename(columns={'DisbursalInMonths':'DisbursalDate'}, inplace=True)

In [15]:
final_df.to_csv("./data/training2.csv",index=False)

In [16]:
final_df['loan_default'].value_counts()

0    182543
1     50611
Name: loan_default, dtype: int64